In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

In [ ]:
from IPython.display import display, HTML

display(HTML("""
<style>
    .widget-label { 
        font-size: 24px !important;
    }
    .slider-value { 
        font-size: 24px !important;
    }
    .widget-hslider .widget-readout {
        font-size: 22px !important;
    }
</style>
"""))

In [59]:
grid_size = 1000
circle_radius = 100

def update_circle(angle_deg):
    angle_rad = np.radians(angle_deg)
    
    circle_x = circle_radius * np.cos(angle_rad)
    circle_y = circle_radius * np.sin(angle_rad)
    
    fig, ax = plt.subplots(1, 2, figsize=(10, 5))

    cur_ax = ax[0]
    cur_ax.set_xlim(-grid_size / 2, grid_size / 2)
    cur_ax.set_ylim(-grid_size / 2, grid_size / 2)
    cur_ax.scatter(circle_x, circle_y, color='red', s=10, label=f"Source: {circle_x:.2f} {circle_y:.2f}")

    r1x = 0
    r1y = -100
    dist_from_source_to_r1 = np.sqrt(((circle_x-r1x)**2) + ((circle_y-r1y)**2))
    transmission_loss_to_r1 = 20*np.log10(dist_from_source_to_r1)
    cur_ax.scatter(r1x, r1y, color='green', s=10, label=f"Receiver #1 {dist_from_source_to_r1:.2f}")

    r1x = 0
    r1y = 100
    dist_from_source_to_r2 = np.sqrt(((circle_x-r1x)**2) + ((circle_y-r1y)**2))
    transmission_loss_to_r2 = 20*np.log10(dist_from_source_to_r2)
    cur_ax.scatter(r1x, r1y, color='blue', s=10, label=f"Receiver #2 {dist_from_source_to_r2:.2f}")
    cur_ax.set_xlabel("X")
    cur_ax.set_ylabel("Y")
    cur_ax.set_title(f"Circle Position (Angle = {angle_deg}°)")
    cur_ax.legend()
    
    cur_ax = ax[1]
    t = np.linspace(0, 10, 1000)
    signal = np.sin(t*np.pi)
    cur_ax.set_title('Transmitted signal')
    cur_ax.plot(t, signal, color='red')
    cur_ax.plot(t, signal/dist_from_source_to_r1, color='blue')
    cur_ax.plot(t, signal/dist_from_source_to_r2, color='green')
    cur_ax.set_xlabel('Time (s)')
    cur_ax.grid(which='both')

    fig.tight_layout()
    plt.show()


In [ ]:

# Create an angle slider (0 to 360 degrees)
angle_slider = widgets.IntSlider(
    value=0, min=0, max=360, step=1, description="Angle (°)", continuous_update=True, style={'description_width': 'initial'}, layout=widgets.Layout(width="600px")
)

# Create interactive widget
interactive_plot = widgets.interactive(update_circle, angle_deg=angle_slider)

# Display the widget
display(interactive_plot)


interactive(children=(IntSlider(value=0, description='Angle (°)', layout=Layout(width='600px'), max=360, style…